In [2]:
# !pip install transformers sentence-transformers openai
# !pip install -U datasets

In [3]:
!nvidia-smi

Sun Dec 28 22:05:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
from datasets import load_dataset

In [9]:
data = load_dataset('rotten_tomatoes')
data

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [6]:
data["train"][1]

{'text': 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
 'label': 1}

In [7]:
from transformers import pipeline

model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device='cuda:0'
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## Model Prediction Without Any Model training 

In [8]:
import numpy as np

In [9]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred = []

for output in tqdm(pipe(KeyDataset(data["test"],"text")), total=len(data['test'])):
    negative_score = output[0]["score"]
    positive_score = output[2]["score"]
    assigment = int(np.argmax([negative_score, positive_score]))
    y_pred.append(assigment)

y_pred

100%|██████████| 1066/1066 [00:11<00:00, 96.90it/s] 


[1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,


In [10]:
y_pred[0]

1

In [15]:
y_true = []
for i in range(len(data['test'])):
    y_true.append(data['test'][i]['label'])

y_true

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [12]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



## Model training by creating embeddings from a Language Model and then using those embeddings to train a classification model

In [13]:
from sentence_transformers import SentenceTransformer

In [14]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [15]:
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [16]:
test_embeddings[0]

array([-9.71327815e-03, -3.21275592e-02, -1.33880023e-02,  2.96829827e-02,
        7.73096131e-03, -8.48837011e-03, -6.43702075e-02,  1.48829166e-02,
       -7.19013624e-03, -2.42439993e-02,  1.47775058e-02,  8.29746295e-03,
       -1.34577695e-02, -4.13124971e-02,  4.50802408e-02, -5.18689640e-02,
        4.17096503e-02,  1.44956578e-02, -2.83935368e-02, -2.17861822e-03,
        3.46153490e-02, -4.08480577e-02,  8.81591067e-03, -2.52877641e-02,
        1.27202580e-02, -5.87383993e-02,  8.27775616e-03,  4.26214375e-02,
       -1.05719222e-02, -3.25730443e-02, -5.16969822e-02,  3.78881842e-02,
       -6.64976938e-03,  2.81441957e-02,  1.79331255e-06,  4.24385304e-03,
        5.64003214e-02, -6.29047304e-03, -1.54684251e-03,  9.61945057e-02,
        9.91486982e-02,  6.68835081e-03, -5.03294542e-03, -2.79494114e-02,
        1.18965618e-02,  3.70403901e-02,  5.92970196e-03, -7.26336166e-02,
       -1.16009861e-02, -5.62669560e-02,  6.47587050e-03, -1.59513857e-02,
       -5.70516400e-02,  

In [17]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=1)
clf.fit(train_embeddings, data["train"]["label"])

LogisticRegression(random_state=1)

In [18]:
y_pred = clf.predict(test_embeddings)
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



In [19]:
from sklearn.svm import SVC

clf = SVC(random_state=1)
clf.fit(train_embeddings, data["train"]["label"])

SVC(random_state=1)

In [20]:
y_pred = clf.predict(test_embeddings)
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.84      0.86      0.85       533
Positive Review       0.86      0.84      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



## Use Cosine similarity of target labels and input texts to determine review lables, without model training

In [21]:
label_embeddings = model.encode(["A negative review","A positive reviw"])

In [22]:
label_embeddings.shape

(2, 768)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(test_embeddings,label_embeddings)
sim_matrix

array([[0.12279499, 0.14179827],
       [0.16302863, 0.17101038],
       [0.04916786, 0.12025397],
       ...,
       [0.00335175, 0.00561365],
       [0.12749438, 0.08086298],
       [0.12168818, 0.0309935 ]], dtype=float32)

In [24]:
y_pred = np.argmax(sim_matrix, axis=1)
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.65      0.81      0.72       533
Positive Review       0.74      0.56      0.64       533

       accuracy                           0.68      1066
      macro avg       0.69      0.68      0.68      1066
   weighted avg       0.69      0.68      0.68      1066



In [25]:
label_embeddings_modified = model.encode(["An extremely negative review","An extremely positive reviw"])
sim_matrix = cosine_similarity(test_embeddings,label_embeddings_modified)
y_pred = np.argmax(sim_matrix, axis=1)
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.75      0.77      0.76       533
Positive Review       0.76      0.74      0.75       533

       accuracy                           0.75      1066
      macro avg       0.75      0.75      0.75      1066
   weighted avg       0.75      0.75      0.75      1066



## Classification using Generative Models

### flan-t5-small

In [27]:
pipe_falant5 = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device="cuda:0"
)
pipe_falant5

Device set to use cuda:0


In [28]:
prompt = "Is the following sentence positive or negative"
data = data.map(lambda x : {"t5": prompt + x["text"]})

data

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [31]:
y_pred = []

for output in tqdm(pipe_falant5(KeyDataset(data['test'],"t5")), total=len(data["test"])):
    response = output[0]['generated_text']
    response = 0 if response=="negative" else 1
    y_pred.append(response)

y_pred

100%|██████████| 1066/1066 [00:54<00:00, 19.71it/s]


[1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,


In [32]:
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.81      0.87      0.84       533
Positive Review       0.86      0.80      0.83       533

       accuracy                           0.83      1066
      macro avg       0.84      0.83      0.83      1066
   weighted avg       0.84      0.83      0.83      1066



## Chat GPT

In [ ]:
# from getpass import getpass
# openai_api_key = getpass()

In [ ]:
import openai

client = openai.OpenAI(api_key='')
client

In [3]:
def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
    """Generate an output based on a prompt and an input document."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

In [4]:
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

'1'

In [10]:
predictions = [chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"])]

100%|██████████| 1066/1066 [06:49<00:00,  2.61it/s]


In [16]:
y_pred = [int(response) for response in predictions]
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=["Negative Review", "Positive Review"]))

                 precision    recall  f1-score   support

Negative Review       0.85      0.97      0.91       533
Positive Review       0.96      0.83      0.89       533

       accuracy                           0.90      1066
      macro avg       0.91      0.90      0.90      1066
   weighted avg       0.91      0.90      0.90      1066



In [17]:
from google.colab import runtime
runtime.unassign()